In [ ]:
# 影片裁減
import ffmpeg

ffmpeg_path = r'C:\Program Files\ffmpeg\bin\ffmpeg.exe'  # 指定ffmpeg路径

def trim_and_crop_video(input_file, output_file, crop_width, crop_height, x, y):
    try:
        (
            ffmpeg
            .input(input_file)
            .filter('crop', crop_width, crop_height, x, y)
            .output(output_file, vcodec='libx264', acodec='aac', preset='veryfast', crf=18, audio_bitrate='192k', map='0:a:0')
            .run(cmd=ffmpeg_path, overwrite_output=True, capture_stdout=True, capture_stderr=True)
        )
        print(f"Trimming and cropping successful: {output_file}")
    except ffmpeg.Error as e:
        print(f"Error occurred: {e.stderr.decode('utf-8')}")
    except Exception as e:
        print(f"Unexpected error: {e}")



# 示例用法
input_file = r"C:\Users\User\Downloads\1.mp4"
output_file = r"C:\Users\User\Downloads\抖音第3集.mp4"

# 原始分辨率 768x480，裁剪到 270x480
crop_width = 270  # 9:16 的宽度
crop_height = 480  # 高度保持不变
x = (768 - crop_width) // 2  # 从水平中心裁剪
y = 0  # 从顶部开始裁剪

trim_and_crop_video(input_file, output_file,  crop_width=crop_width, crop_height=crop_height, x=x, y=y)


In [ ]:
import ffmpeg

ffmpeg_path = r'C:\Program Files\ffmpeg\bin\ffmpeg.exe'  # 指定ffmpeg路径

def get_video_duration(input_file):
    try:
        probe = ffmpeg.probe(input_file)
        video_info = next(stream for stream in probe['streams'] if stream['codec_type'] == 'video')
        return float(video_info['duration'])
    except ffmpeg.Error as e:
        print(f"Error occurred: {e.stderr.decode('utf-8')}")
    except Exception as e:
        print(f"Unexpected error: {e}")
        return None

def trim_and_crop_video(input_file, output_file, crop_width, crop_height, x, y, start_time):
    try:
        # 获取视频总时长
        video_duration = get_video_duration(input_file)
        if video_duration is None:
            print("Unable to get video duration.")
            return
        
        # 设置 end_time 为视频总长度
        end_time = video_duration

        (
            ffmpeg
            .input(input_file, ss=start_time, to=end_time)  # 从 start_time 秒开始处理，到视频结束
            .filter('crop', crop_width, crop_height, x, y)
            .output(output_file, vcodec='libx264', acodec='aac', preset='veryfast', crf=18, audio_bitrate='192k', map='0:a:0')
            .run(cmd=ffmpeg_path, overwrite_output=True, capture_stdout=True, capture_stderr=True)
        )
        print(f"Trimming and cropping successful: {output_file}")
    except ffmpeg.Error as e:
        print(f"Error occurred: {e.stderr.decode('utf-8')}")
    except Exception as e:
        print(f"Unexpected error: {e}")

# 示例用法
input_file = r"C:\Users\User\Downloads\1.mp4"
output_file = r"C:\Users\User\Downloads\抖音第3集.mp4"

# 原始分辨率 768x480，左半部分宽度是 768 / 2 = 384
original_width = 2304
original_height = 1440

# 左半部分的宽度
left_half_width = original_width // 2

# 保留左边部分的右边4/5
crop_width = left_half_width * 3 // 4  # 左半部分的3/4
crop_height = original_height
x = left_half_width // 4  # 裁掉左边部分的1/4，开始裁剪的x坐标
y = 0  # 从顶部开始裁剪

start_time = 0  # 从第20秒开始裁剪

trim_and_crop_video(input_file, output_file, crop_width=crop_width, crop_height=crop_height, x=x, y=y, start_time=start_time)


In [ ]:
#將影片轉成多張圖片
import cv2
import os

# 創建資料夾如果它不存在
output_folder = 'movie'
os.makedirs(output_folder, exist_ok=True)

# 打開視頻文件
vidcap = cv2.VideoCapture(r"C:\Users\User\Downloads\finish.mp4")
frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = vidcap.get(cv2.CAP_PROP_FPS)

# 計算影片的總長度（秒）
duration = frame_count / fps

# 指定開始和結束的時間點 (秒)
start_time = 0
end_time = duration  # 到第20秒結束

# 設置視頻的當前幀位置到開始時間點
vidcap.set(cv2.CAP_PROP_POS_MSEC, start_time * 1000)

fps = int(vidcap.get(cv2.CAP_PROP_FPS))  # 獲取影片的幀率
frame_count = 0
success, image = vidcap.read()

while success:
    current_time = vidcap.get(cv2.CAP_PROP_POS_MSEC) / 1000  # 當前幀的時間 (秒)
    if current_time > end_time:
        break

    cv2.imwrite(os.path.join(output_folder, f"frame_{frame_count:04d}.jpg"), image)  # 保存幀為 jpg 文件
    success, image = vidcap.read()  # 讀取下一幀
    frame_count += 1


In [ ]:
import re
import os
import subprocess
import requests

def extract_youtube_links(file_path):
    """從指定檔案中提取所有 YouTube 連結 (包括影片和播放清單)"""
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # 正則表達式匹配所有 http/https 開頭的 YouTube 影片和播放清單連結
    youtube_links = re.findall(r'(https?://(?:www\.)?(?:youtube\.com|youtu\.be)[^\s]+)', content)

    return youtube_links

def is_video_available(url):
    """檢查影片是否能夠訪問"""
    try:
        response = requests.head(url, allow_redirects=True)
        if response.status_code == 200:
            return True
    except requests.RequestException as e:
        print(f"檢查影片時出現錯誤: {str(e)}")
    return False

def download_video_or_playlist(link):
    """使用 yt-dlp 下載單個影片或整個播放清單"""
    print(f"正在下載: {link}")
    
    # 設定 yt-dlp 的下載選項
    ydl_command = [
        'yt-dlp',
        '-f', 'best',  # 下載最高畫質
        '--yes-playlist',  # 允許下載整個播放清單
        '--age-limit', '18',  # 允許下載18禁影片
        link
    ]
    
    try:
        # 使用 subprocess 執行 yt-dlp 命令來下載影片或播放清單
        result = subprocess.run(ydl_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if result.returncode == 0:
            print(f"下載成功: {link}")
        else:
            print(f"下載失敗，錯誤訊息:\n{result.stderr.decode('utf-8')}")
    
    except Exception as e:
        print(f"下載影片或播放清單時發生錯誤: {str(e)}")

def download_videos(links):
    """嘗試下載所有連結，跳過無法下載的影片"""
    if not links:
        print("沒有找到 YouTube 連結。")
        return

    for link in links:
        if is_video_available(link):
            download_video_or_playlist(link)
        else:
            print(f"無法下載或影片無效，跳過: {link}")

if __name__ == "__main__":
    # 提供記事本文件的路徑
    file_path = 'C:/Users/User/桌面/note.txt'  # 修改為你的記事本路徑

    # 1. 從記事本中提取所有 YouTube 連結（影片和播放清單）
    youtube_links = extract_youtube_links(file_path)

    # 2. 下載所有找到的 YouTube 影片或播放清單，跳過無效影片
    download_videos(youtube_links)


ffmpeg -i "C:\Users\User\Desktop\audio\6月11日 下午8點27分.mp3" -f lavfi -i color=c=black:s=1280x720:d=60 -c:v libx264 -c:a aac -b:a 192k -shortest "C:/Users/User/Desktop/video/6月11日 下午8點27分.mp4"

In [ ]:
#TODO轉換錄音到影片
import os
import subprocess
import re
from tqdm import tqdm
import time

# 配置参数
input_audio_folder = 'C:\\Users\\User\\Desktop\\audio'
output_video_folder = 'C:/Users/User/Desktop/video'
background_color = 'black'
video_resolution = '1280x720'  # 视频分辨率
timeout_seconds = 420  # 超过7分钟

# 确保输出文件夹存在
os.makedirs(output_video_folder, exist_ok=True)

def get_audio_duration(audio_file):
    """获取音频文件的持续时间（秒）。"""
    ffmpeg_path = r'C:\Program Files\ffmpeg\bin\ffmpeg.exe'
    try:
        result = subprocess.run(
            [ffmpeg_path, '-i', audio_file],
            stderr=subprocess.PIPE,
            text=True,
            encoding='utf-8'
        )
        duration_match = re.search(r'Duration: (\d+):(\d+):(\d+\.\d+)', result.stderr)
        if duration_match:
            h, m, s = map(float, duration_match.groups())
            return h * 3600 + m * 60 + s
        else:
            raise ValueError("无法获取音频时长。")
    except Exception as e:
        print(f"获取 {audio_file} 时长出错: {e}")
        raise

def parse_ffmpeg_progress(line):
    """解析 FFmpeg 的进度输出，返回已处理的时间（秒）。"""
    match = re.search(r'time=(\d+):(\d+):(\d+\.\d+)', line)
    if match:
        h, m, s = map(float, match.groups())
        return h * 3600 + m * 60 + s
    return None

def process_audio_file(audio_file):
    """处理单个音频文件并生成视频。"""
    file_name, _ = os.path.splitext(os.path.basename(audio_file))
    output_video_file = os.path.join(output_video_folder, f"{file_name}.mp4")
    
    try:
        audio_duration = get_audio_duration(audio_file)
    except Exception as e:
        print(f"处理 {audio_file} 时出错: {e}")
        return
    
    ffmpeg_path = r'C:\Program Files\ffmpeg\bin\ffmpeg.exe'
    cmd = [
        ffmpeg_path,
        '-f', 'lavfi', 
        '-i', f'color=c={background_color}:s={video_resolution}:d={audio_duration}',
        '-i', audio_file,
        '-c:v', 'libx264',
        '-preset', 'medium',
        '-crf', '30',
        '-c:a', 'aac',
        '-b:a', '192k',
        '-shortest',
        output_video_file
    ]
    
    print(f'Processing file: {file_name}')
    with tqdm(total=audio_duration, desc=f'Converting {file_name}', leave=False) as pbar:
        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, encoding='utf-8')

        start_time = time.time()  # 记录开始时间
        while True:
            line = process.stderr.readline()
            if not line:
                break
            processed_time = parse_ffmpeg_progress(line)
            if processed_time is not None:
                pbar.update(processed_time - pbar.n)
                
            # 检查是否超过超时时间
            if time.time() - start_time > timeout_seconds:
                print(f"{file_name} 处理超时，跳过。")
                process.terminate()  # 终止处理
                break

        process.wait()
        pbar.n = audio_duration
        pbar.close()

    # 在成功生成视频文件后删除音频文件
    if os.path.exists(output_video_file):
        print(f"删除音频文件: {audio_file}")
        os.remove(audio_file)

def batch_process_audio_files(input_folder):
    """批量处理文件夹中的音频文件。"""
    audio_files = [os.path.join(input_folder, file_name) for file_name in os.listdir(input_folder)
                   if file_name.lower().endswith(('.aac', '.m4a', '.wav', '.mp3'))]

    for audio_file in audio_files:
        process_audio_file(audio_file)

# 批量处理音频文件
batch_process_audio_files(input_audio_folder)


In [ ]:
#TODO:上傳錄音黨
import os
import pickle
import time  # 用於延遲
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

# 憑證和範圍
SCOPES = ['https://www.googleapis.com/auth/youtube.upload']
CLIENT_SECRETS_FILE = r"C:\Users\User\Desktop\Programm\pecoCTF\client_secret.json"
EXCEL_FILE_PATH = r"C:\Users\User\Desktop\分割程式.xlsx"
VIDEO_FOLDER_PATH = r"C:\Users\User\Desktop\video"

# 高亮填充的顏色（螢光色背景）
HIGHLIGHT_FILL = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

# 影片說明模板
VIDEO_DESCRIPTION_TEMPLATE = """
錄{recording_time}
若你是本影片中出現的人物，請盡快與我聯絡，非營利用途。
"""

def get_authenticated_service():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return build('youtube', 'v3', credentials=creds)

def upload_video(video_file, title, description, category_id, tags):
    youtube = get_authenticated_service()

    request_body = {
        'snippet': {
            'title': title,
            'description': description,
            'tags': tags,
            'categoryId': category_id
        },
        'status': {
            'privacyStatus': 'public'  # 可設為 'private' 或 'unlisted'
        }
    }

    media_file = MediaFileUpload(video_file)

    response_upload = youtube.videos().insert(
        part="snippet,status",
        body=request_body,
        media_body=media_file
    ).execute()

    print(f"影片已上傳，ID: {response_upload.get('id')}")

def normalize_filename(filename):
    """
    去除檔案名稱中的空格，並返回標準化後的名稱。確保 filename 是字串。
    """
    return str(filename).replace(" ", "")

def extract_first_12_digits(filename):
    """
    從檔案名稱中取出前12個數字。確保 filename 是字串。
    """
    normalized_name = normalize_filename(filename)
    return normalized_name[:12]  # 返回前12個字符

def load_progress(sheet):
    """
    從 Excel 的 F2 欄位讀取進度。如果 F2 欄為空，則返回 97 作為起始行。
    """
    progress_cell = sheet['F2']
    if progress_cell.value:
        return int(progress_cell.value)
    return 97

def save_progress(sheet, row):
    """
    將當前進度保存到 Excel 的 F2 欄位中，並保存下一個行號以避免重複處理。
    """
    sheet['F2'] = row + 1  # 下一個行號

def main():
    # 加載 Excel 文件
    wb = load_workbook(EXCEL_FILE_PATH)
    sheet = wb.active

    # 從 Excel 的 F2 欄位加載上次的進度
    start_row = load_progress(sheet)

    # 遍歷 Excel 中的每一行，從上次保存的行開始
    for row in sheet.iter_rows(min_row=start_row, max_row=sheet.max_row):
        video_title_cell = row[2]  # "姓名" 欄 (第C欄)
        video_file_name_cell = row[0]  # "複製用" 欄
        recording_time_cell = row[4]  # "複製用2" 欄 (第E欄)
        comparison_name_cell = row[1]  # "比大小" 欄 (第B欄)

        # 判斷是否為 "新的錄音"
        if "新的錄音" in video_title_cell.value:
            video_file_name_to_find = video_title_cell.value + ".mp4"
        # 判斷是否為 "新錄音"，使用 "比大小" 欄位，且比較檔案名稱的前12個數字
        elif "新錄音" in video_title_cell.value:
            normalized_filename = extract_first_12_digits(comparison_name_cell.value)
            found_file = None
            for video_file in os.listdir(VIDEO_FOLDER_PATH):
                if extract_first_12_digits(video_file) == normalized_filename:
                    found_file = video_file
                    break

            if found_file:
                video_file_name_to_find = found_file
            else:
                print(f"未找到匹配的影片：比大小 = {comparison_name_cell.value}")
                break  # 如果找不到檔案，則停止程式
        else:
            video_file_name_to_find = video_file_name_cell.value + ".mp4"

        # 查找影片檔案
        video_file_path = os.path.join(VIDEO_FOLDER_PATH, video_file_name_to_find)

        # 如果影片檔案存在，則上傳
        if os.path.exists(video_file_path):
            video_title = "[生活心理學] " + video_title_cell.value
            video_description = VIDEO_DESCRIPTION_TEMPLATE.format(recording_time=recording_time_cell.value)

            print(f"正在上傳影片：{video_file_path}")

            # 上傳影片
            upload_video(
                video_file=video_file_path,
                title=video_title,
                description=video_description,
                category_id="22",  # 22 表示 'People & Blogs' 類別
                tags=["tag1", "tag2"]
            )

            # 標記這一行的 "姓名" 欄位為螢光色背景
            video_title_cell.fill = HIGHLIGHT_FILL

            # 保存當前進度（下一個行號）到 F2
            save_progress(sheet, row[0].row)

            # 保存 Excel 文件
            wb.save(EXCEL_FILE_PATH)

            # 延遲 10 秒鐘
            time.sleep(10)
        else:
            print(f"未找到影片：{video_file_name_to_find}")
            break  # 如果找不到檔案，則停止程式

if __name__ == '__main__':
    main()


In [12]:
i=input()
for t in i:
    print('/' if t=='\\' else t,end='')

In [ ]:
import os
import pickle
import google.auth
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from googleapiclient.http import MediaFileUpload

# YouTube OAuth 2.0 credentials file
CLIENT_SECRETS_FILE = "client_secret2.json"
SCOPES = ["https://www.googleapis.com/auth/youtube.upload"]
API_SERVICE_NAME = "youtube"
API_VERSION = "v3"

# File to keep track of uploaded videos
UPLOADED_VIDEOS_FILE = "uploaded_videos.txt"

# Define a set of video file extensions
VIDEO_EXTENSIONS = {'.mp4', '.avi', '.mov', '.mkv', '.flv', '.wmv', '.webm'}

def authenticate():
    # Authenticate and create YouTube API client
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        CLIENT_SECRETS_FILE, SCOPES)
    
    # Start a local server to handle the OAuth callback
    credentials = flow.run_local_server(port=0)
    
    youtube = googleapiclient.discovery.build(
        API_SERVICE_NAME, API_VERSION, credentials=credentials)
    
    return youtube


def load_uploaded_videos():
    # Load list of previously uploaded videos from the file
    if os.path.exists(UPLOADED_VIDEOS_FILE):
        with open(UPLOADED_VIDEOS_FILE, "r") as file:
            return set(line.strip() for line in file.readlines())
    return set()


def save_uploaded_video(video_file):
    # Save the uploaded video file name to the file
    with open(UPLOADED_VIDEOS_FILE, "a") as file:
        file.write(f"{video_file}\n")


def upload_video(youtube, video_file, title, description, category="22", privacy_status="private"):
    # Upload a video to YouTube
    print(f"Uploading {video_file}...")
    
    request_body = {
        "snippet": {
            "title": title,
            "description": description,
            "categoryId": category
        },
        "status": {
            "privacyStatus": privacy_status
        }
    }

    media = MediaFileUpload(video_file, chunksize=-1, resumable=True)
    
    request = youtube.videos().insert(
        part="snippet,status",
        body=request_body,
        media_body=media
    )

    response = None
    while response is None:
        status, response = request.next_chunk()
        if status:
            print(f"Uploaded {int(status.progress() * 100)}%")
    
    print(f"Video {video_file} uploaded successfully!")
    return response


def is_video_file(file_name):
    # Check if the file has a video extension
    return os.path.splitext(file_name)[1].lower() in VIDEO_EXTENSIONS


def upload_videos_in_folder(youtube, folder_path):
    # Get list of videos in the folder and check for duplicates
    uploaded_videos = load_uploaded_videos()
    files_in_folder = os.listdir(folder_path)

    for file_name in files_in_folder:
        full_path = os.path.join(folder_path, file_name)
        
        # Only process video files
        if is_video_file(file_name) and file_name not in uploaded_videos:
            # Customize the title and description as needed
            title = f"Video {file_name}"
            description = f"Description for {file_name}"

            upload_video(youtube, full_path, title, description)
            save_uploaded_video(file_name)
        elif not is_video_file(file_name):
            print(f"Skipping non-video file: {file_name}")
        else:
            print(f"{file_name} has already been uploaded, skipping.")


if __name__ == "__main__":
    youtube_client = authenticate()
    folder_path = r"D:\電腦錄影"  # Update to your video folder path
    upload_videos_in_folder(youtube_client, folder_path)


In [4]:
import cv2

# 載入影片
video_path = r"C:\Users\User\Downloads\apple.mp4"
cap = cv2.VideoCapture(video_path)

# 取得影片的寬度、高度與幀率
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# 定義輸出的影片格式與編碼器
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('mosaic_output.mp4', fourcc, fps, (width, height))

# 設定馬賽克的格數 (4x4馬賽克，共16格)
mosaic_size = 4
block_width = width // mosaic_size
block_height = height // mosaic_size

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 將每一幀劃分成 mosaic_size x mosaic_size 的馬賽克格子
    for i in range(mosaic_size):
        for j in range(mosaic_size):
            # 計算每個馬賽克格子的左上角和右下角座標
            x_start = i * block_width
            y_start = j * block_height
            x_end = x_start + block_width
            y_end = y_start + block_height

            # 提取該區域的像素
            roi = frame[y_start:y_end, x_start:x_end]

            # 計算該區域的平均顏色
            avg_color = roi.mean(axis=(0, 1))

            # 將該區域填充為平均顏色，實現馬賽克效果
            frame[y_start:y_end, x_start:x_end] = avg_color

    # 將處理後的幀寫入輸出的影片
    out.write(frame)
# 釋放資源
cap.release()
out.release()
cv2.destroyAllWindows()


In [ ]:
import os
import yt_dlp

# 播放清單的 URL
playlist_url = 'https://youtube.com/playlist?list=PLZ9OZPYCX824R3egW5EES5VYPc2gbbypG&si=tWYyy1aQnbxUtpYA'


# 指定下載目錄
download_path ='D:best'

# 確保目錄存在，如果不存在則建立
if not os.path.exists(download_path):
    os.makedirs(download_path)

# yt-dlp 的選項設置
ydl_opts = {
    'format': 'bestaudio/best',  # 只下載音訊，選擇最高音質
    'outtmpl': os.path.join(download_path, '%(title)s.%(ext)s'),  # 文件命名格式
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',  # 可以選擇 mp3 格式
        'preferredquality': '192',  # 音質設置，192kbps
    }],
    'noplaylist': False,  # 允許下載播放清單
    'playlist_items': ','.join(map(str, range(0,1000))),  # 指定下載的影片編號範圍
    'age_limit': None,  # 允許所有年齡的內容
    'postprocessor_args': [
        '-loglevel', 'error'  # 僅顯示錯誤訊息
    ],
    'noprogress': True,  # 下載時不顯示進度
    'ignoreerrors': True,  # 忽略錯誤，繼續下載其他影片
}

# 使用 yt-dlp 下載播放清單
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([playlist_url])
